# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
#Import your libraries
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


# Challenge 1 - Import and Describe the Dataset

#### In this challenge we will use the `austin_weather` data. 

#### First, import it into a data frame called `austin`. 

In [2]:
# Your code here
data=pd.read_csv("austin_weather.csv")


#### Next, describe the dataset you have loaded: 
- Look at the variables and their types
- Examine the descriptive statistics of the numeric variables 
- Look at the first five rows of all variables to evaluate the categorical variables as well

In [3]:
# Your code here
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Date                        1319 non-null   object
 1   TempHighF                   1319 non-null   int64 
 2   TempAvgF                    1319 non-null   int64 
 3   TempLowF                    1319 non-null   int64 
 4   DewPointHighF               1319 non-null   object
 5   DewPointAvgF                1319 non-null   object
 6   DewPointLowF                1319 non-null   object
 7   HumidityHighPercent         1319 non-null   object
 8   HumidityAvgPercent          1319 non-null   object
 9   HumidityLowPercent          1319 non-null   object
 10  SeaLevelPressureHighInches  1319 non-null   object
 11  SeaLevelPressureAvgInches   1319 non-null   object
 12  SeaLevelPressureLowInches   1319 non-null   object
 13  VisibilityHighMiles         1319 non-null   obje

#### Given the information you have learned from examining the dataset, write down three insights about the data in a markdown cell below

#### Your Insights:

1. There are 21 variables in the dataset. 3 of them are numeric and the rest contain some text.

2. The average temperature in Austin ranged between around 70 degrees F and around 93 degrees F. The highest temperature observed during this period was 107 degrees F and the lowest was 19 degrees F.

3. When we look at the head function, we see that a lot of variables contain numeric data even though these columns are of object type. This means we might have to do some data cleansing.


#### Let's examine the DewPointAvgF variable by using the `unique()` function to list all unique values in this dataframe.

Describe what you find in a markdown cell below the code. What did you notice? What do you think made Pandas to treat this column as *object* instead of *int64*? 

In [4]:
# Your code here
data.DewPointAvgF.unique()

array(['49', '36', '27', '28', '40', '39', '41', '26', '42', '22', '48',
       '32', '8', '11', '45', '55', '61', '37', '47', '25', '23', '20',
       '33', '30', '29', '17', '14', '13', '54', '59', '15', '24', '34',
       '35', '57', '50', '53', '60', '46', '56', '51', '31', '38', '62',
       '43', '63', '64', '67', '66', '58', '70', '68', '65', '69', '71',
       '72', '-', '73', '74', '21', '44', '52', '12', '75', '76', '18'],
      dtype=object)

### Your observation here
    - Because the values are not different for each cell, but a series of values that repeat in some cells. For this case there are 66 unique values for DewPoinAvgF.
    - Could be also, that the values were entered initially as text instead of numbers


The following is a list of columns misrepresented as `object`. Use this list to convert the columns to numeric using the `pandas.to_numeric` function in the next cell. If you encounter errors in converting strings to numeric values, you need to catch those errors and force the conversion by supplying `errors='coerce'` as an argument for `pandas.to_numeric`. Coercing will replace non-convertable elements with `NaN` which represents an undefined numeric value. This makes it possible for us to conveniently handle missing values in subsequent data processing.

*Hint: you may use a loop to change one column at a time but it is more efficient to use `apply`.*

In [5]:
wrong_type_columns = ['DewPointHighF', 'DewPointAvgF', 'DewPointLowF', 'HumidityHighPercent', 
                      'HumidityAvgPercent', 'HumidityLowPercent', 'SeaLevelPressureHighInches', 
                      'SeaLevelPressureAvgInches' ,'SeaLevelPressureLowInches', 'VisibilityHighMiles',
                      'VisibilityAvgMiles', 'VisibilityLowMiles', 'WindHighMPH', 'WindAvgMPH', 
                      'WindGustMPH', 'PrecipitationSumInches']

In [6]:
data[wrong_type_columns]=data[wrong_type_columns].apply(pd.to_numeric, errors="coerce")

#### Check if your code has worked by printing the data types again. You should see only two `object` columns (`Date` and `Events`) now. All other columns should be `int64` or `float64`.

In [7]:
# Your code here
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        1319 non-null   object 
 1   TempHighF                   1319 non-null   int64  
 2   TempAvgF                    1319 non-null   int64  
 3   TempLowF                    1319 non-null   int64  
 4   DewPointHighF               1312 non-null   float64
 5   DewPointAvgF                1312 non-null   float64
 6   DewPointLowF                1312 non-null   float64
 7   HumidityHighPercent         1317 non-null   float64
 8   HumidityAvgPercent          1317 non-null   float64
 9   HumidityLowPercent          1317 non-null   float64
 10  SeaLevelPressureHighInches  1316 non-null   float64
 11  SeaLevelPressureAvgInches   1316 non-null   float64
 12  SeaLevelPressureLowInches   1316 non-null   float64
 13  VisibilityHighMiles         1307 

# Challenge 2 - Handle the Missing Data

#### Now that we have fixed the type mismatch, let's address the missing data.

By coercing the columns to numeric, we have created `NaN` for each cell containing characters. We should choose a strategy to address these missing data.

The first step is to examine how many rows contain missing data.

We check how much missing data we have by applying the `.isnull()` function to our dataset. To find the rows with missing data in any of its cells, we apply `.any(axis=1)` to the function. `austin.isnull().any(axis=1)` will return a column containing true if the row contains at least one missing value and false otherwise. Therefore we must subset our dataframe with this column. This will give us all rows with at least one missing value. 

#### In the next cell, identify all rows containing at least one missing value. Assign the dataframes with missing values to a variable called `missing_values`.

In [8]:
# Your code here
data["missing_values"]=data.isnull().any(axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        1319 non-null   object 
 1   TempHighF                   1319 non-null   int64  
 2   TempAvgF                    1319 non-null   int64  
 3   TempLowF                    1319 non-null   int64  
 4   DewPointHighF               1312 non-null   float64
 5   DewPointAvgF                1312 non-null   float64
 6   DewPointLowF                1312 non-null   float64
 7   HumidityHighPercent         1317 non-null   float64
 8   HumidityAvgPercent          1317 non-null   float64
 9   HumidityLowPercent          1317 non-null   float64
 10  SeaLevelPressureHighInches  1316 non-null   float64
 11  SeaLevelPressureAvgInches   1316 non-null   float64
 12  SeaLevelPressureLowInches   1316 non-null   float64
 13  VisibilityHighMiles         1307 

There are multiple strategies to handle missing data. Below lists the most common ones data scientists use:

* Removing all rows or all columns containing missing data. This is the simplest strategy. It may work in some cases but not others.

* Filling all missing values with a placeholder value. 
    * For categorical data, `0`, `-1`, and `9999` are some commonly used placeholder values. 
    * For continuous data, some may opt to fill all missing data with the mean. This strategy is not optimal since it can increase the fit of the model.

* Filling the values using some algorithm. 

#### In our case, we will use a hybrid approach which is to first remove the data that contain most missing values then fill in the rest of the missing values with the *linear interpolation* algorithm.

#### Next, count the number of rows of `austin` and `missing_values`.

In [9]:
# Your code here
print(data.shape[0])
print(data.missing_values.value_counts()[1])

1319
136


#### Calculate the ratio of missing rows to total rows

In [10]:
# Your code here
print(f"ratio of True-missing_values: {(data.missing_values.value_counts()[1]/data.shape[0])*100}")
print(f"ratio of False-missing_values: {(data.missing_values.value_counts()[0]/data.shape[0])*100}")

ratio of True-missing_values: 10.310841546626232
ratio of False-missing_values: 89.68915845337376


As you can see, there is a large proportion of missing data (over 10%). Perhaps we should evaluate which columns have the most missing data and remove those columns. For the remaining columns, we will perform a linear approximation of the missing data.

We can find the number of missing rows in each column using the `.isna()` function. We then chain the `.sum` function to the `.isna()` function and find the number of missing rows per column

In [11]:
# Your code here
data.isna().sum()

Date                            0
TempHighF                       0
TempAvgF                        0
TempLowF                        0
DewPointHighF                   7
DewPointAvgF                    7
DewPointLowF                    7
HumidityHighPercent             2
HumidityAvgPercent              2
HumidityLowPercent              2
SeaLevelPressureHighInches      3
SeaLevelPressureAvgInches       3
SeaLevelPressureLowInches       3
VisibilityHighMiles            12
VisibilityAvgMiles             12
VisibilityLowMiles             12
WindHighMPH                     2
WindAvgMPH                      2
WindGustMPH                     4
PrecipitationSumInches        124
Events                          0
missing_values                  0
dtype: int64

#### As you can see from the output, the majority of missing data is in one column called `PrecipitationSumInches`. What's the number of missing values in this column in ratio to its total number of rows?

In [12]:
# Your code here
print(f"ratio of missing_values in PrecipitationSumInches: {(data.isna().sum()[19]/data.shape[0])*100}")

ratio of missing_values in PrecipitationSumInches: 9.401061410159212


Almost 10% data missing! Therefore, we prefer to remove this column instead of filling its missing values. 

#### Remove this column from `austin` using the `.drop()` function. Use the `inplace=True` argument.

*Hints:*

* By supplying `inplace=True` to `drop()`, the original dataframe object will be changed in place and the function will return `None`. In contrast, if you don't supply `inplace=True`, which is equivalent to supplying `inplace=False` because `False` is the default value, the original dataframe object will be kept and the function returns a copy of the transformed dataframe object. In the latter case, you'll have to assign the returned object back to your variable.

* Also, since you are dropping a column instead of a row, you'll need to supply `axis=1` to `drop()`.

[Reference for `pandas.DataFrame.drop`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)

In [13]:
# Your code here 
data.drop("PrecipitationSumInches", axis=1, inplace=True)

# Print `austin` to confirm the column is indeed removed

data

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,Events,missing_values
0,2013-12-21,74,60,45,67.0,49.0,43.0,93.0,75.0,57.0,...,29.68,29.59,10.0,7.0,2.0,20.0,4.0,31.0,"Rain , Thunderstorm",False
1,2013-12-22,56,48,39,43.0,36.0,28.0,93.0,68.0,43.0,...,30.13,29.87,10.0,10.0,5.0,16.0,6.0,25.0,,False
2,2013-12-23,58,45,32,31.0,27.0,23.0,76.0,52.0,27.0,...,30.49,30.41,10.0,10.0,10.0,8.0,3.0,12.0,,False
3,2013-12-24,61,46,31,36.0,28.0,21.0,89.0,56.0,22.0,...,30.45,30.30,10.0,10.0,7.0,12.0,4.0,20.0,,False
4,2013-12-25,58,50,41,44.0,40.0,36.0,86.0,71.0,56.0,...,30.33,30.27,10.0,10.0,7.0,10.0,2.0,16.0,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71.0,67.0,61.0,82.0,54.0,25.0,...,29.97,29.88,10.0,10.0,10.0,12.0,5.0,21.0,,False
1315,2017-07-28,105,91,76,71.0,64.0,55.0,87.0,54.0,20.0,...,29.90,29.81,10.0,10.0,10.0,14.0,5.0,20.0,,False
1316,2017-07-29,107,92,77,72.0,64.0,55.0,82.0,51.0,19.0,...,29.86,29.79,10.0,10.0,10.0,12.0,4.0,17.0,,False
1317,2017-07-30,106,93,79,70.0,68.0,63.0,69.0,48.0,27.0,...,29.91,29.87,10.0,10.0,10.0,13.0,4.0,20.0,,False


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        1319 non-null   object 
 1   TempHighF                   1319 non-null   int64  
 2   TempAvgF                    1319 non-null   int64  
 3   TempLowF                    1319 non-null   int64  
 4   DewPointHighF               1312 non-null   float64
 5   DewPointAvgF                1312 non-null   float64
 6   DewPointLowF                1312 non-null   float64
 7   HumidityHighPercent         1317 non-null   float64
 8   HumidityAvgPercent          1317 non-null   float64
 9   HumidityLowPercent          1317 non-null   float64
 10  SeaLevelPressureHighInches  1316 non-null   float64
 11  SeaLevelPressureAvgInches   1316 non-null   float64
 12  SeaLevelPressureLowInches   1316 non-null   float64
 13  VisibilityHighMiles         1307 

#### Next we will perform linear interpolation of the missing data.

This means that we will use a linear algorithm to estimate the missing data. Linear interpolation assumes that there is a straight line between the points and the missing point will fall on that line. This is a good enough approximation for weather related data. Weather related data is typically a time series. Therefore, we do not want to drop rows from our data if possible. It is prefereable to estimate the missing values rather than remove the rows. However, if you have data from a single point in time, perhaps a better solution would be to remove the rows. 

If you would like to read more about linear interpolation, you can do so [here](https://en.wikipedia.org/wiki/Linear_interpolation).

In the following cell, use the `.interpolate()` function on the entire dataframe. This time pass the `inplace=False` argument to the function and assign the interpolated dataframe to a new variable called `austin_fixed` so that we can compare with `austin`.

In [15]:
# Your code here
data_fixed=data.interpolate()

#### Check to make sure `austin_fixed` contains no missing data. Also check `austin` - it still contains missing data.

In [16]:
# Your code here
data_fixed.isna().sum()

Date                          0
TempHighF                     0
TempAvgF                      0
TempLowF                      0
DewPointHighF                 0
DewPointAvgF                  0
DewPointLowF                  0
HumidityHighPercent           0
HumidityAvgPercent            0
HumidityLowPercent            0
SeaLevelPressureHighInches    0
SeaLevelPressureAvgInches     0
SeaLevelPressureLowInches     0
VisibilityHighMiles           0
VisibilityAvgMiles            0
VisibilityLowMiles            0
WindHighMPH                   0
WindAvgMPH                    0
WindGustMPH                   0
Events                        0
missing_values                0
dtype: int64

In [17]:
data.isna().sum()

Date                           0
TempHighF                      0
TempAvgF                       0
TempLowF                       0
DewPointHighF                  7
DewPointAvgF                   7
DewPointLowF                   7
HumidityHighPercent            2
HumidityAvgPercent             2
HumidityLowPercent             2
SeaLevelPressureHighInches     3
SeaLevelPressureAvgInches      3
SeaLevelPressureLowInches      3
VisibilityHighMiles           12
VisibilityAvgMiles            12
VisibilityLowMiles            12
WindHighMPH                    2
WindAvgMPH                     2
WindGustMPH                    4
Events                         0
missing_values                 0
dtype: int64

# Challenge 3 - Processing the `Events` Column

#### Our dataframe contains one true text column - the Events column. We should evaluate this column to determine how to process it.

Use the `value_counts()` function to evaluate the contents of this column

In [18]:
# Your code here:
data_fixed.Events.value_counts()

Events
                             903
Rain                         192
Rain , Thunderstorm          137
Fog , Rain , Thunderstorm     33
Fog                           21
Thunderstorm                  17
Fog , Rain                    14
Rain , Snow                    1
Fog , Thunderstorm             1
Name: count, dtype: int64

Reading the values of `Events` and reflecting what those values mean in the context of data, you realize this column indicates what weather events had happened in a particular day.

#### What is the largest number of events happened in a single day? Enter your answer in the next cell.

## Your answer:
    - Rain is the event that have most happened


#### We want to transform the string-type `Events` values to the numbers. This will allow us to apply machine learning algorithms easily.

How? We will create a new column for each type of events (i.e. *Rain*, *Snow*, *Fog*, *Thunderstorm*. In each column, we use `1` to indicate if the corresponding event happened in that day and use `0` otherwise.

Below we provide you a list of all event types. Loop the list and create a dummy column with `0` values for each event in `austin_fixed`. To create a new dummy column with `0` values, simply use `austin_fixed[event] = 0`.

In [19]:
event_list = ['Snow', 'Fog', 'Rain', 'Thunderstorm']

# Your code here
for item in event_list:
    data_fixed[item]=0

# Print your new dataframe to check whether new columns have been created:

data_fixed

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,Events,missing_values,Snow,Fog,Rain,Thunderstorm
0,2013-12-21,74,60,45,67.0,49.0,43.0,93.0,75.0,57.0,...,2.0,20.0,4.0,31.0,"Rain , Thunderstorm",False,0,0,0,0
1,2013-12-22,56,48,39,43.0,36.0,28.0,93.0,68.0,43.0,...,5.0,16.0,6.0,25.0,,False,0,0,0,0
2,2013-12-23,58,45,32,31.0,27.0,23.0,76.0,52.0,27.0,...,10.0,8.0,3.0,12.0,,False,0,0,0,0
3,2013-12-24,61,46,31,36.0,28.0,21.0,89.0,56.0,22.0,...,7.0,12.0,4.0,20.0,,False,0,0,0,0
4,2013-12-25,58,50,41,44.0,40.0,36.0,86.0,71.0,56.0,...,7.0,10.0,2.0,16.0,,True,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71.0,67.0,61.0,82.0,54.0,25.0,...,10.0,12.0,5.0,21.0,,False,0,0,0,0
1315,2017-07-28,105,91,76,71.0,64.0,55.0,87.0,54.0,20.0,...,10.0,14.0,5.0,20.0,,False,0,0,0,0
1316,2017-07-29,107,92,77,72.0,64.0,55.0,82.0,51.0,19.0,...,10.0,12.0,4.0,17.0,,False,0,0,0,0
1317,2017-07-30,106,93,79,70.0,68.0,63.0,69.0,48.0,27.0,...,10.0,13.0,4.0,20.0,,False,0,0,0,0


#### Next, populate the actual values in the dummy columns of  `austin_fixed`.

You will check the *Events* column. If its string value contains `Rain`, then the *Rain* column should be `1`. The same for `Snow`, `Fog`, and `Thunderstorm`.

*Hints:*

* Use [`pandas.Series.str.contains()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html) to create the value series of each new column.

* What if the values you populated are booleans instead of numbers? You can cast the boolean values to numbers by using `.astype(int)`. For instance, `pd.Series([True, True, False]).astype(int)` will return a new series with values of `[1, 1, 0]`.

In [20]:
# Your code here
final=[]
for item in data_fixed.Events:
    item_series=pd.Series(item)
    result=[]
    for types in event_list:
        values=item_series.str.contains(types)
        result.append(values[0].astype(int))
    final.append(result)

snow_=[]
fog_=[]
rain_=[]
Thunderstorm_=[]

for item in final:
    snow_.append(item[0])
    fog_.append(item[1])
    rain_.append(item[2])
    Thunderstorm_.append(item[3])    
    
data_fixed["Snow"]=snow_
data_fixed["Fog"]=fog_
data_fixed["Rain"]=rain_
data_fixed["Thunderstorm"]=Thunderstorm_

#### Print out `austin_fixed` to check if the event columns are populated with the intended values

In [21]:
# Your code here
data_fixed

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,Events,missing_values,Snow,Fog,Rain,Thunderstorm
0,2013-12-21,74,60,45,67.0,49.0,43.0,93.0,75.0,57.0,...,2.0,20.0,4.0,31.0,"Rain , Thunderstorm",False,0,0,1,1
1,2013-12-22,56,48,39,43.0,36.0,28.0,93.0,68.0,43.0,...,5.0,16.0,6.0,25.0,,False,0,0,0,0
2,2013-12-23,58,45,32,31.0,27.0,23.0,76.0,52.0,27.0,...,10.0,8.0,3.0,12.0,,False,0,0,0,0
3,2013-12-24,61,46,31,36.0,28.0,21.0,89.0,56.0,22.0,...,7.0,12.0,4.0,20.0,,False,0,0,0,0
4,2013-12-25,58,50,41,44.0,40.0,36.0,86.0,71.0,56.0,...,7.0,10.0,2.0,16.0,,True,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71.0,67.0,61.0,82.0,54.0,25.0,...,10.0,12.0,5.0,21.0,,False,0,0,0,0
1315,2017-07-28,105,91,76,71.0,64.0,55.0,87.0,54.0,20.0,...,10.0,14.0,5.0,20.0,,False,0,0,0,0
1316,2017-07-29,107,92,77,72.0,64.0,55.0,82.0,51.0,19.0,...,10.0,12.0,4.0,17.0,,False,0,0,0,0
1317,2017-07-30,106,93,79,70.0,68.0,63.0,69.0,48.0,27.0,...,10.0,13.0,4.0,20.0,,False,0,0,0,0


#### If your code worked correctly, now we can drop the `Events` column as we don't need it any more.

In [22]:
# Your code here
data_fixed.drop("Events", axis=1, inplace=True)

# Challenge 4 - Processing The `Date` Column

The `Date` column is another non-numeric field in our dataset. A value in that field looks like `'2014-01-06'` which consists of the year, month, and day connected with hyphens. One way to convert the date string to numerical is using a similar approach as we used for `Events`, namely splitting the column into numerical `Year`, `Month`, and `Day` columns. In this challenge we'll show you another way which is to use the Python `datetime` library's `toordinal()` function. Depending on what actual machine learning analysis you will conduct, each approach has its pros and cons. Our goal today is to practice data preparation so we'll skip the discussion here.

Here you can find the [reference](https://docs.python.org/3/library/datetime.html) and [example](https://stackoverflow.com/questions/39846918/convert-date-to-ordinal-python) for `toordinal`. The basic process is to first convert the string to a `datetime` object using `datetime.datetime.strptime`, then convert the `datetime` object to numerical using `toordinal`.

#### In the cell below, convert the `Date` column values from string to numeric values using `toordinal()`.

In [23]:
# Your code here
data_fixed["Date"]=data_fixed["Date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
data_fixed["Date"]=data_fixed["Date"].apply(lambda x: x.toordinal())

#### Print `austin_fixed` to check your `Date` column.

In [24]:
data_fixed.head(5)

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,missing_values,Snow,Fog,Rain,Thunderstorm
0,735223,74,60,45,67.0,49.0,43.0,93.0,75.0,57.0,...,7.0,2.0,20.0,4.0,31.0,False,0,0,1,1
1,735224,56,48,39,43.0,36.0,28.0,93.0,68.0,43.0,...,10.0,5.0,16.0,6.0,25.0,False,0,0,0,0
2,735225,58,45,32,31.0,27.0,23.0,76.0,52.0,27.0,...,10.0,10.0,8.0,3.0,12.0,False,0,0,0,0
3,735226,61,46,31,36.0,28.0,21.0,89.0,56.0,22.0,...,10.0,7.0,12.0,4.0,20.0,False,0,0,0,0
4,735227,58,50,41,44.0,40.0,36.0,86.0,71.0,56.0,...,10.0,7.0,10.0,2.0,16.0,True,0,0,0,0


# Challenge 5 - Sampling and Holdout Sets

#### Now that we have processed the data for machine learning, we will separate the data to test and training sets.

We first train the model using only the training set. We check our metrics on the training set. We then apply the model to the test set and check our metrics on the test set as well. If the metrics are significantly more optimal on the training set, then we know we have overfit our model. We will need to revise our model to ensure it will be more applicable to data outside the test set.

#### In the next cells we will separate the data into a training set and a test set using the `train_test_split()` function in scikit-learn.

When using `scikit-learn` for machine learning, we first separate the data to predictor and response variables. This is the standard way of passing datasets into a model in `scikit-learn`. The `scikit-learn` will then find out whether the predictors and responses fit the model.

In the next cell, assign the `TempAvgF` column to `y` and the remaining columns to `X`. Your `X` should be a subset of `austin_fixed` containing the following columns: 

```['Date',
 'TempHighF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'Snow',
 'Fog',
 'Rain',
 'Thunderstorm']```
 
 Your `y` should be a subset of `austin_fixed` containing one column `TempAvgF`.

In [25]:
# Your code here:
y=data_fixed["TempAvgF"]
X=data_fixed[['Date',  'TempHighF',  'TempLowF',  'DewPointHighF',  'DewPointAvgF',  'DewPointLowF',  'HumidityHighPercent',  'HumidityAvgPercent',  'HumidityLowPercent',  'SeaLevelPressureHighInches',  'SeaLevelPressureAvgInches',  'SeaLevelPressureLowInches',  'VisibilityHighMiles',  'VisibilityAvgMiles',  'VisibilityLowMiles',  'WindHighMPH',  'WindAvgMPH',  'WindGustMPH',  'Snow',  'Fog',  'Rain',  'Thunderstorm']]

In the next cell, import `train_test_split` from `sklearn.model_selection`

In [26]:
#Your code here:
from sklearn.model_selection import train_test_split

Now that we have split the data to predictor and response variables and imported the `train_test_split()` function, split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`. 80% of the data should be in the training set and 20% in the test set. `train_test_split()` reference can be accessed [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).


Enter your code in the cell below:

In [27]:
#Your code here:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [28]:
y_train.head()

1261    80
49      51
674     69
1191    75
549     84
Name: TempAvgF, dtype: int64

In [29]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [30]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [31]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns, index=X_train.index )
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns, index=X_test.index)

In [32]:
from sklearn.neighbors import KNeighborsRegressor

In [33]:
knn = KNeighborsRegressor()

In [34]:
knn.fit(X_train_norm, y_train)


KNeighborsRegressor()

In [35]:
#Test Score R2:
knn.score(X_test_norm, y_test)

0.9384406156332097

In [36]:
#Train Score R2:
knn.score(X_train_norm, y_train)

0.9583683212716031

In [37]:
test_pred=knn.predict(X_test_norm)
test_pred

array([81.2, 58.4, 67. , 86.6, 75.4, 81.4, 52.6, 77.2, 72.2, 87.8, 81.4,
       60.4, 89.4, 79. , 79.2, 59.4, 69. , 56.2, 51.4, 70.4, 70.6, 49. ,
       67.2, 64.4, 50.6, 63.4, 72.2, 62.2, 75.6, 74.2, 54.8, 88.2, 41. ,
       54.2, 67.6, 77.2, 50.4, 82.2, 62.6, 82.2, 77.8, 51.6, 72.8, 77.4,
       69.2, 82. , 73.2, 77.8, 80.6, 75.2, 73.6, 71.4, 84. , 89.4, 83.2,
       42.4, 87.6, 85. , 62.2, 67.6, 88.2, 75.6, 68.8, 79.6, 85.8, 84.4,
       74.6, 64. , 70.2, 78.6, 82. , 56.2, 51.8, 88.2, 75.6, 82.6, 68.4,
       70.2, 70. , 57.8, 86.6, 87. , 88.6, 84.8, 81.6, 79.8, 68.2, 64.6,
       47.8, 43. , 62.2, 59.2, 83.8, 83. , 60.8, 89.4, 36.6, 74.6, 89.4,
       84.2, 75.6, 52. , 79.6, 66.6, 76. , 57.4, 69.2, 68.6, 52. , 89.2,
       75. , 84.2, 52.2, 63.2, 41.6, 73.4, 68. , 87. , 89.8, 63.6, 78. ,
       52. , 86.2, 78. , 81.4, 85.8, 72.4, 67.6, 89.2, 72.8, 64.2, 68.6,
       78.8, 73. , 78.4, 68. , 71.8, 89.4, 72. , 72.2, 78.2, 85.8, 48.2,
       59.8, 69.6, 74.8, 79.6, 77.8, 85. , 82. , 76

In [38]:
y_test.values

array([79, 56, 64, 83, 76, 80, 53, 74, 75, 90, 84, 67, 90, 78, 79, 64, 70,
       60, 54, 71, 67, 48, 59, 63, 53, 61, 71, 65, 75, 75, 51, 89, 35, 51,
       67, 78, 50, 78, 64, 83, 75, 48, 75, 78, 68, 82, 64, 77, 79, 77, 74,
       77, 83, 91, 86, 34, 85, 83, 61, 67, 90, 73, 73, 76, 85, 82, 76, 65,
       72, 76, 83, 60, 49, 89, 81, 84, 65, 66, 73, 57, 84, 83, 88, 86, 82,
       83, 69, 64, 47, 29, 59, 54, 81, 83, 62, 89, 29, 72, 88, 83, 76, 53,
       80, 68, 74, 51, 64, 69, 57, 90, 72, 84, 43, 56, 38, 76, 70, 85, 89,
       66, 74, 48, 86, 77, 80, 85, 73, 55, 88, 74, 70, 63, 77, 73, 76, 67,
       65, 91, 72, 71, 79, 86, 39, 57, 67, 80, 79, 78, 85, 87, 78, 84, 89,
       40, 92, 69, 87, 85, 70, 63, 69, 74, 52, 86, 42, 70, 68, 54, 78, 83,
       66, 54, 85, 76, 54, 74, 58, 82, 89, 77, 86, 70, 87, 82, 50, 84, 75,
       51, 52, 74, 54, 79, 78, 89, 42, 73, 82, 62, 49, 79, 69, 79, 70, 72,
       78, 86, 71, 73, 65, 89, 84, 64, 82, 66, 87, 87, 86, 74, 38, 89, 78,
       60, 86, 85, 72, 87

#### Congratulations! Now you have finished the preparation of the dataset!

### Trying BagginRegressor

In [39]:
bagging_reg = BaggingRegressor(KNeighborsRegressor(n_neighbors=10),
                               n_estimators=50,
                               max_samples = 500)

In [40]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=10), max_samples=500,
                 n_estimators=50)

In [41]:
pred=bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 2.896810606060605
RMSE 4.05319852185601
R2 score 0.9168992682763796


In [42]:
print("R2 score", bagging_reg.score(X_train_norm, y_train))

R2 score 0.9247723775504944


# Bonus Challenge 1

#### While the above is the common practice to prepare most datasets, when it comes to time series data, we sometimes do not want to randomly select rows from our dataset.

This is because many time series algorithms rely on observations having equal time distances between them. In such cases, we typically select the majority of rows as the test data and the last few rows as the training data. We don't use `train_test_split()` to select the train/test data because it returns random selections.

In the following cell, compute the number of rows that account for 80% of our data and round it to the next integer. Assign this number to `ts_rows`.

In [43]:
# Your code here:
ts_rows=round(data_fixed.shape[0]*0.80,0)
ts_rows

1055.0

Assign the first `ts_rows` rows of `X` to `X_ts_train` and the remaining rows to `X_ts_test`.

In [45]:
# Your code here:
X_ts_train=X.iloc[:int(ts_rows),:]
X_ts_test=X.iloc[1055:, :]

Assign the first `ts_rows` rows of `y` to `y_ts_train` and the remaining rows to `y_ts_test`.

In [49]:
# Your code here:
y_ts_train=y.iloc[:int(ts_rows)]
y_ts_test=y.iloc[1055:]

In [50]:
knn_bonus = KNeighborsRegressor(n_neighbors=3)

In [51]:
knn_bonus.fit(X_ts_train, y_ts_train)

KNeighborsRegressor(n_neighbors=3)

In [52]:
#Test Score R2:
knn_bonus.score(X_test, y_test)

0.8561413896249243

In [53]:
#Train Score R2:
knn_bonus.score(X_train, y_train)

0.8712533254233141

In [54]:
knn_bonus.predict(X_test)

array([78.66666667, 56.        , 74.66666667, 84.33333333, 76.66666667,
       79.33333333, 54.33333333, 73.        , 77.33333333, 89.66666667,
       83.66666667, 69.        , 76.66666667, 79.33333333, 77.66666667,
       63.33333333, 69.66666667, 61.66666667, 55.66666667, 69.66666667,
       68.33333333, 50.33333333, 65.66666667, 65.        , 53.66666667,
       72.33333333, 69.33333333, 74.66666667, 74.66666667, 76.33333333,
       53.        , 88.33333333, 36.33333333, 49.        , 74.66666667,
       80.        , 51.66666667, 78.33333333, 65.        , 81.33333333,
       76.33333333, 66.33333333, 74.33333333, 75.33333333, 68.33333333,
       82.        , 65.66666667, 78.33333333, 79.        , 76.33333333,
       74.        , 76.66666667, 84.33333333, 76.66666667, 87.66666667,
       35.66666667, 85.        , 83.        , 66.33333333, 66.66666667,
       88.33333333, 71.66666667, 71.33333333, 76.        , 85.        ,
       83.33333333, 76.66666667, 75.        , 74.66666667, 77.33